<h1>CNN Using Raw Spectra as Inputs</h1>

<p>This notebook is used to train and explore a CNN model that classifies minerals by taking in raw spectra</p>

In [1]:
import numpy as np
import pandas as pd
import cnnmodel_raw as cnn_raw
from tensorflow.keras.models import Model


<h1>Use this space below for fiddling with model training</h1>

<p>If you need to retrain the model, be sure to restart the runtime</p>

<h3>Define Hyperparameters below</h3>

In [2]:
learning_rate=0.001
batch_size=500
drop_rate=0.55
epochs=5

#don't change the line below
hyperparameters=[learning_rate,batch_size,drop_rate,epochs]

In [3]:
#this string is prepended to all output files for unique identification
id_value='v1_test_21_12_9'

#training data file path
fin_path=r'Data/Raw Data/Single/'

#output file path for the trained model data
mout_path=r'Model Data/CNN Model/'

#proportion of the training data allocated to the dev set - a float x: 0<x<1
dev_size=0.2

#random seed used for train/dev split to ensure repeatable outcomes. None 
#provides a psuedorandom value that won't be repeatable
r_state=1

#fast - True for doing quick training that does not output probability weights
#for each sample during training. False for doing deep analysis and tracking
#note if fast is true, model will train in <5 min while false may be >30 min
fast=True

#threshold - float in the range [0.0,1.0) determines the discrimination threshold
threshold=0.5


In [4]:
'''train the model'''
cnn_model,train_history,X_val,y_val=cnn_raw.raw_cnn_model(fin_path,mout_path,dev_size,r_state,hyperparameters,
                          fast,id_value,threshold=threshold)

<h2><b>Test the model</b></h2>

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from pprint import pprint
from IPython.display import clear_output
from sklearn.metrics import roc_curve

train_accuracy = train_history.history['sparse_categorical_accuracy']
train_accuracy_df = pd.DataFrame(train_accuracy)
#train_accuracy_df.to_csv(train_accuracy_path, index = False, header = False)

val_accuracy = train_history.history['val_sparse_categorical_accuracy']
val_accuracy_df = pd.DataFrame(val_accuracy)
#val_accuracy_df.to_csv(val_accuracy_path, index = False, header = False)


plt.plot(train_accuracy, label = 'Training Accuracy')
plt.plot(val_accuracy, label = 'Validation Accuracy')
plt.title('Accuracy Curves')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc = 'best')
plt.show()

In [ ]:
train_loss = train_history.history['loss']
train_loss_df = pd.DataFrame(train_loss)
#train_loss_df.to_csv(train_loss_path, index = False, header = False)

val_loss = train_history.history['val_loss']
val_loss_df = pd.DataFrame(val_loss)
#val_loss_df.to_csv(val_loss_path, index = False, header = False)

plt.plot(train_loss, label = 'Training Loss')
plt.plot(val_loss, label = 'Validation Loss')
plt.title('Loss Curves')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc = 'best')
plt.show()

In [ ]:
def plot_confusion_matrix(y_true, y_pred, title, path):
    import matplotlib.pyplot as plt  
    labels = np.arange(15)
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    cm_df = pd.DataFrame(cm, columns = labels, index = labels)
    #cm_df.to_csv(path, index = True, header = True)
    cm_df.columns.name = 'Predicted'
    cm_df.index.name = 'Actual'
    fig, ax = plt.subplots(figsize=(13,10))         # Sample figsize in inches
    sn.heatmap(cm_df, annot = True, cmap = 'Blues', fmt = 'd', cbar = False)
    plt.title(title)
    plt.show()


In [ ]:
val_cm_path=r'Model Data/CNN Model/Raw/val_confusion_matrix_'+id_value+r'.csv'
plot_confusion_matrix(y_val, np.argmax(cnn_model.predict(X_val), axis = 1), 'Validation Confusion Matrix', val_cm_path)

In [ ]:
testin_path=r'Data/Raw Data/Mixed/Test Set/'
test_cm_path=r'Model Data/CNN Model/Raw/test_confusion_matrix_'+id_value+r'.csv'

import helper as h
#build test set, drops samples with Null values
test_df=h.dfbuilder(testin_path,split_df=False,dev_size=0.2,r_state=1,raw=True)


#separates features and labels
y_test=test_df[test_df.columns[-1]]
X_test=test_df.drop(test_df.columns[-1],axis=1).values


plot_confusion_matrix(y_test, np.argmax(cnn_model.predict(X_test), axis = 1), 'Test Confusion Matrix', test_cm_path) 
print(cnn_model.predict(X_test))

In [ ]:
print(classification_report(y_test, np.argmax(cnn_model.predict(X_test), axis = -1)))

cr = classification_report(y_test, np.argmax(cnn_model.predict(X_test), axis = -1), output_dict = True)
cr_df = pd.DataFrame(cr).transpose()
#cr_df.to_csv(cr_path, index = True, header = True)